## Kütüphanelerin Yüklenmesi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.svm import LinearSVR

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import pickle
import json


In [ ]:
## Bazı ilçelerde ev özellikleri değişebilmektedir. 
## Örneğin bir ilçede 6+1 odalı evler varken başka bir ilçede hiç 6+1 odalı ev olmayabilir.
## Bu nedenle her ilçede hangi özelliklerin bulunduğunun ayrıştırılması gerekir.

ILCE_LIST = ["esenyurt", "besiktas", "pendik", "kucukcekmece", "bahcelievler", "uskudar", "kadikoy", "sariyer", "bakirkoy", "sisli"]
roomCount = {}
buildingAge = {}
heatingType = {}


for ilce in ILCE_LIST:  
  houses_df = pd.read_csv("/content/drive/MyDrive/dataset/" + ilce + ".csv")
  roomCount[ilce] = houses_df['Oda Sayısı'].unique().tolist()
  buildingAge[ilce] = houses_df['Bina Yaşı'].unique().tolist()
  heatingType[ilce] = houses_df['Isıtma Tipi'].unique().tolist()

with open('roomCount.json', 'w') as fp:
    json.dump(roomCount, fp)

with open('buildingAge.json', 'w') as fp:
    json.dump(buildingAge, fp)

with open('heatingType.json', 'w') as fp:
    json.dump(heatingType, fp)

  


In [ ]:
## Mobil uygulamadan gelen kullanıcı girişini model girdisine dönüştürmek için her ilçenin default giriş değeri oluşturulmuştur.

ILCE_LIST = ["esenyurt", "besiktas", "pendik", "kucukcekmece", "bahcelievler", "uskudar", "kadikoy", "sariyer", "bakirkoy", "sisli"]
DEFAULT_INPUTS = {}
for ilce in ILCE_LIST:  
  houses_df = pd.read_csv("/content/" + ilce + ".csv")
  houses_df = houses_df[houses_df['Kira'] < 50000]
  houses_df = houses_df[houses_df['Kira'] > 1000]
  houses_df = houses_df.reset_index(drop=True)
  
  oda_hashmap = {'6+1': 8, '2+0': 2,'2+2': 6, '1.5+1': 2, '2.5+1': 4, '2+1': 4,'3+1': 5,'5+2': 9,'Stüdyo': 1,'1+1': 2,'3+2': 7,'1 Oda': 1,'4+1': 6,'5+1': 7,'4+2': 8,'8+ Oda': 13,'7+2': 11,'6+2': 10, '3.5+1': 5}
  bina_hashmap = {'21 Ve Üzeri': 25, '5-10': 7, '11-15': 13, '16-20': 18, '4': 4, '0 (Yeni)': 0, '1': 1, '3': 3, '2': 2}
  balkon_hashmap = {'Yok': -1, 'Var': 1}
  esya_hashmap = {'Boş': -1, 'Eşyalı': 1}
  site_hashmap = {'Evet': 1, 'Hayır': -1}
  houses_df = houses_df.replace({'Oda Sayısı': oda_hashmap})
  houses_df = houses_df.replace({'Bina Yaşı': bina_hashmap})
  houses_df = houses_df.replace({'Balkon Durumu': balkon_hashmap})
  houses_df = houses_df.replace({'Eşya Durumu': esya_hashmap})
  houses_df = houses_df.replace({'Site İçerisinde': site_hashmap})
  houses_df = pd.get_dummies(houses_df)
  houses_df.pop('Kira')
  print(houses_df.columns)
  DEFAULT_INPUTS[ilce] = houses_df[:1]

pickle.dump(DEFAULT_INPUTS, open('DEFAULT_INPUTS', 'wb'))


Index(['Alan', 'Oda Sayısı', 'Banyo Sayısı', 'Bina Yaşı', 'Balkon Durumu',
       'Eşya Durumu', 'Site İçerisinde', 'Isıtma Tipi_Doğalgaz Sobalı',
       'Isıtma Tipi_Kat Kaloriferi', 'Isıtma Tipi_Klimalı',
       'Isıtma Tipi_Kombi Doğalgaz', 'Isıtma Tipi_Merkezi (Pay Ölçer)',
       'Isıtma Tipi_Merkezi Doğalgaz', 'Isıtma Tipi_Sobalı'],
      dtype='object')
Index(['Alan', 'Oda Sayısı', 'Banyo Sayısı', 'Bina Yaşı', 'Balkon Durumu',
       'Eşya Durumu', 'Site İçerisinde', 'Isıtma Tipi_Kombi Doğalgaz',
       'Isıtma Tipi_Merkezi (Pay Ölçer)', 'Isıtma Tipi_Merkezi Doğalgaz',
       'Isıtma Tipi_Yerden Isıtma'],
      dtype='object')
Index(['Alan', 'Oda Sayısı', 'Banyo Sayısı', 'Bina Yaşı', 'Balkon Durumu',
       'Eşya Durumu', 'Site İçerisinde', 'Isıtma Tipi_Doğalgaz Sobalı',
       'Isıtma Tipi_Isıtma Yok', 'Isıtma Tipi_Kombi Doğalgaz',
       'Isıtma Tipi_Merkezi (Pay Ölçer)', 'Isıtma Tipi_Merkezi Doğalgaz',
       'Isıtma Tipi_Sobalı', 'Isıtma Tipi_Yerden Isıtma'],
      dtype='ob

In [ ]:
## Bu kısımda veri setleri ile her bir ilçe için 4 adet model eğitilmiştir.

import pickle 

ILCE_LIST = ["esenyurt", "besiktas", "pendik", "kucukcekmece", "bahcelievler", "uskudar", "kadikoy", "sariyer", "bakirkoy", "sisli"]

for ilce in ILCE_LIST:
  houses_df = pd.read_csv("/content/" + ilce + ".csv")
  houses_df.head()
  houses_df = houses_df[houses_df['Kira'] < 50000]
  houses_df = houses_df[houses_df['Kira'] > 1000]
  houses_df = houses_df.reset_index(drop=True)
  oda_hashmap = {'6+1': 8, '2+0': 2,'2+2': 6, '1.5+1': 2, '2.5+1': 4, '2+1': 4,'3+1': 5,'5+2': 9,'Stüdyo': 1,'1+1': 2,'3+2': 7,'1 Oda': 1,'4+1': 6,'5+1': 7,'4+2': 8,'8+ Oda': 13,'7+2': 11,'6+2': 10, '3.5+1': 5}
  bina_hashmap = {'21 Ve Üzeri': 25, '5-10': 7, '11-15': 13, '16-20': 18, '4': 4, '0 (Yeni)': 0, '1': 1, '3': 3, '2': 2}
  balkon_hashmap = {'Yok': -1, 'Var': 1}
  esya_hashmap = {'Boş': -1, 'Eşyalı': 1}
  site_hashmap = {'Evet': 1, 'Hayır': -1}
  houses_df = houses_df.replace({'Oda Sayısı': oda_hashmap})
  houses_df = houses_df.replace({'Bina Yaşı': bina_hashmap})
  houses_df = houses_df.replace({'Balkon Durumu': balkon_hashmap})
  houses_df = houses_df.replace({'Eşya Durumu': esya_hashmap})
  houses_df = houses_df.replace({'Site İçerisinde': site_hashmap})
  houses_df.head()
  houses_df = pd.get_dummies(houses_df)
  houses_df_train, houses_df_test = train_test_split(houses_df, 
                                     train_size = 0.9, 
                                     test_size = 0.1, 
                                     random_state = 10)
  X_train = houses_df_train ## Kira harici kategoriler
  y_train = X_train.pop('Kira') ## Kira kategorisi
  X_test = houses_df_test
  y_test = X_test.pop('Kira')
  models = [LinearRegression(),
              RandomForestRegressor(n_estimators=100, max_features='sqrt'),
              KNeighborsRegressor(n_neighbors=1),
              SVR(kernel='linear'),
              ]
  trained_models=[]
  for model in models:
    reg_model = model.fit(X_train, y_train)
    trained_models.append(reg_model)
    y_pred = reg_model.predict(X_train)
    final_score = r2_score(y_train, y_pred)
    
    print(model, final_score)

  pickle.dump(trained_models, open("/content/drive/MyDrive/models2/" + ilce, 'wb'))

LinearRegression() 0.16944886434200424
RandomForestRegressor(max_features='sqrt') 0.8321900336078512
KNeighborsRegressor(n_neighbors=1) 0.9626601371159621
SVR(kernel='linear') 0.08365435252398434
LinearRegression() 0.5469989219520641
RandomForestRegressor(max_features='sqrt') 0.9145756288953486
KNeighborsRegressor(n_neighbors=1) 0.9369804295045662
SVR(kernel='linear') 0.29497122065975634
LinearRegression() 0.8131707013212472
RandomForestRegressor(max_features='sqrt') 0.8811636932237815
KNeighborsRegressor(n_neighbors=1) 0.994593769831273
SVR(kernel='linear') 0.3004181976917597
LinearRegression() 0.6433705403584877
RandomForestRegressor(max_features='sqrt') 0.9438576488261188
KNeighborsRegressor(n_neighbors=1) 0.9966517354914586
SVR(kernel='linear') 0.2961577943848617
LinearRegression() 0.5077581354365606
RandomForestRegressor(max_features='sqrt') 0.9034862379317073
KNeighborsRegressor(n_neighbors=1) 0.9662608645664289
SVR(kernel='linear') 0.30667663236586507
LinearRegression() 0.759977